In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)


# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os


import warnings
warnings.filterwarnings("ignore")

import janestreet


env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set



import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

print("XGBoost version:", xgb.__version__)


train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv') ##, nrows=500)
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

print('train shape is {}'.format(train.shape))
print('features shape is {}'.format(features.shape))
print('example_test shape is {}'.format(example_test.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

#train.head()

# I have taked this cell from https://www.kaggle.com/jazivxt/the-market-is-reactive
# And https://www.kaggle.com/drcapa/jane-street-market-prediction-starter-xgb

train = train[train['weight'] != 0]

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')


X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

X_train = X_train.fillna(-999)


y = train['action'].value_counts().values

# trace2 = go.Bar(
#          x=x ,
#          y=y,
#          marker=dict(
#                       color=y,
#                       colorscale = 'Viridis',
#                       reversescale = True
#                   ),
#          name="Imbalance",
#      )
# layout = dict(
#          title="Data imbalance - action",
#          #width = 900, height = 500,
#          xaxis=go.layout.XAxis(
#                   automargin=True),
#          yaxis=dict(
#                       showgrid=False,
#                       showline=False,
#                       showticklabels=True,
#               #         domain=[0, 0.85],
#                   ),
#     )
# fig1 = go.Figure(data=[trace2], layout=layout)
# iplot(fig1)


# del x, y, train

# The training part taked from here https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s

#import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# clf = xgb.XGBClassifier(
#         n_estimators=500,
#         max_depth=13,
#         learning_rate=0.05,
#         subsample=0.9,
#         colsample_bytree=0.7,
#         missing=-999,
#         random_state=2020,
#         tree_method='gpu_hist'  # THE MAGICAL PARAMETER
#     )

##%time clf.fit(X_train, y_train)

### begin linear regression

#y_train,X_train


# Load the diabetes dataset
#diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True) ## two arrays

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
# diabetes_X_train = diabetes_X[:-20]
# diabetes_X_test = diabetes_X[-20:]

# # Split the targets into training/testing sets
# diabetes_y_train = diabetes_y[:-20]
# diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
##regr.fit(diabetes_X_train, diabetes_y_train)

regr.fit(X_train, y_train)

# Make predictions using the testing set
# diabetes_y_pred = regr.predict(diabetes_X_test)

# # The coefficients
# print('Coefficients: \n', regr.coef_)
# # The mean squared error
# print('Mean squared error: %.2f'
#             % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# # The coefficient of determination: 1 is perfect prediction
# print('Coefficient of determination: %.2f'
#             % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
# plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
# plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()
### end linear regression

for (test_df, sample_prediction_df) in iter_test:
  X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
  X_test.fillna(-999, inplace = True)
  y_preds = regr.predict(X_test)
  print(len(y_preds))
  ##y_preds = clf.predict(X_test)
  if y_preds >= 0.004:
    sample_prediction_df.action = 1
  else:
    sample_prediction_df.action = 0          
  env.predict(sample_prediction_df)
